# Time Travel Query   

Db2 offers a rich set of temporal data management capabilities collectively known as Db2 Time Travel Query. Db2 Time Travel Query is a catchall for a number of related technologies that include temporal tables, temporal queries, temporal updates, and temporal constraints, along with other temporal functionality. All of these capabilities are available in all Db2 editions, so there are no licensing considerations when you want to use this feature.

If you wanted to implement some kind of time travel capability before Db2 10, you had to create triggers and complex application logic to trace data changes, manage effective dates, and enforce temporal constraints. 

Through simple declarative SQL statements, you can instruct Db2 to maintain a history of database changes or track effective business dates automatically, eliminating the need for such logic to be hand-coded into triggers, stored procedures, or application code: this is a tremendous benefit.

## Stock Trade Application   

This example illustrates a simplistic stock trade system that tracks the historical value of stocks over time. 

There are a number of functions, tables, and procedures that are used to run this example:

- `STOCKS` - This table contains the base stock information
- `STOCKS_HISTORY` - This table keeps any updates and deletes that may have occured 
- `GENERATE_UPDATES` - This function is used to generate INSERTS/UPDATES/DELETES of stocks

## Stock Table   

The `STOCK` table contains six columns:

- `STOCK KEY` - a three character code representing the stock value
- `STOCK PRICE` - the price of the stock (at the current time)
- `QUANTITY` - the number of stocks that were sold

An additional three columns are required to track the values of the stocks over time.

- `SYS_START` - When the row was valid
- `SYS_END`   - Last date the row was valid
- `TRANS_ID`  - Transaction ID used when multiple tables are involved in an update

Finally we need to tell Db2 that the period time is the combination of the `SYS_START` and `SYS_END` columns

### Load Db2 Extensions and Connect to Db2

In [ ]:
%run db2.ipynb
%sql CONNECT TO SAMPLE USER DB2INST1 USING db2inst1

### Create the Stock table

In [ ]:
%%sql -q
DROP TABLE TTQ.STOCKS;

CREATE TABLE TTQ.STOCKS
  (
  STOCK      CHAR(3) NOT NULL,
  PRICE      INT NOT NULL,
  QUANTITY   INT NOT NULL,
  SYS_START  TIMESTAMP(12) GENERATED ALWAYS AS ROW BEGIN NOT NULL,
  SYS_END    TIMESTAMP(12) GENERATED ALWAYS AS ROW END NOT NULL,
  TRANS_ID   TIMESTAMP(12) GENERATED ALWAYS AS TRANSACTION START ID IMPLICITLY HIDDEN,
  PERIOD SYSTEM_TIME (SYS_START, SYS_END)
  );

### History Table   

The History table is an exact copy of the base table and is used to track any `UPDATES` or `DELETES` to the base table. INSERTs are always in the base table so there is no need to track then in the history table.

**Note:** You could also add the syntax `WITH RESTRICT ON DROP` to prevent the user from dropping the base table without first doing something with the history table.

In [ ]:
%%sql -q
DROP TABLE TTQ.STOCKS_HISTORY;

CREATE TABLE TTQ.STOCKS_HISTORY
  LIKE TTQ.STOCKS;

## Linking Base Table to History Table   

At this point we can link the two tables together. The `ALTER` command will tell Db2 that the `STOCKS_HISTORY` table is to be used to track `UPDATES` and `DELETES` on the base table.

In [ ]:
%%sql
ALTER TABLE TTQ.STOCKS ADD VERSIONING
  USE HISTORY TABLE TTQ.STOCKS_HISTORY;

### Generate Data for the STOCK table   

This initial insert sets up the `STOCKS` that will be tracked in this system. The actual `STOCK` symbols are derived from airport codes around the world and are not related to real stocks in any way. Before we create this table we will keep track of the current time so we can go "back in time" in our queries.

In [ ]:
%%sql
CREATE OR REPLACE VARIABLE TTQ.CREATE_TIME TIMESTAMP;

SET TTQ.CREATE_TIME = CURRENT TIMESTAMP;

INSERT INTO TTQ.STOCKS(STOCK, PRICE, QUANTITY)
  VALUES
     ('YYZ',100,50),
     ('GRU',32,133),
     ('LHR',45,40),
     ('FRA',98,89);
       
SELECT * FROM TTQ.STOCKS;

## Querying the HISTORY table   

The history table (`TTQ.STOCKS_HISTORY`) is available for direct queries. You may want to directly query this table if you need to look for particular transactions, or if you want to prune the entries in this table. The number of rows in this table can grow over time so you may find it necessary to delete values from it occassionally.

The following transaction deletes the YYZ stock from the list and then selects the values from the history table to show that it is there.

In [ ]:
%%sql
DELETE FROM TTQ.STOCKS WHERE 
  STOCK = 'YYZ';
  
SELECT * FROM TTQ.STOCKS_HISTORY;

Updating a row will also result in an entry in the History table. The first portion of this command gets the current time so we can see what the value of a row was before and after an update.

In [ ]:
%%sql
CREATE OR REPLACE VARIABLE TTQ.BEFORE_UPDATE TIMESTAMP;

SET TTQ.BEFORE_UPDATE = CURRENT TIMESTAMP;

UPDATE TTQ.STOCKS
  SET PRICE = 150
  WHERE STOCK = 'GRU';

List the current value of the GRU stock.

In [ ]:
%%sql
SELECT STOCK, PRICE FROM TTQ.STOCKS 
  WHERE STOCK = 'GRU';

What was the value of the stock prior to the update? 

In [ ]:
%%sql
SELECT STOCK, PRICE FROM TTQ.STOCKS
  FOR SYSTEM_TIME AS OF TTQ.BEFORE_UPDATE
  WHERE STOCK = 'GRU';

Update the GRU record, LHR, and FRA records.

In [ ]:
%%sql
UPDATE TTQ.STOCKS 
  SET PRICE = 76
  WHERE STOCK IN ('GRU','LHR','FRA');

How many changes were there to the GRU stock since we created the table?

In [ ]:
%%sql
SELECT COUNT(*) FROM TTQ.STOCKS
  FOR SYSTEM_TIME BETWEEN TTQ.CREATE_TIME AND CURRENT TIMESTAMP
  WHERE STOCK = 'GRU';

What is the maximum value that the GRU stock had since we created the table?

In [ ]:
%%sql
SELECT MAX(PRICE) FROM TTQ.STOCKS
  FOR SYSTEM_TIME BETWEEN TTQ.CREATE_TIME AND CURRENT TIMESTAMP
  WHERE STOCK = 'GRU';

What is the maximum ad minimum price of all stocks and the number of trades since we created the table?

In [ ]:
%%sql
SELECT STOCK, MIN(PRICE) AS LOW, MAX(PRICE) AS HIGH, COUNT(*) AS TRADES FROM TTQ.STOCKS
  FOR SYSTEM_TIME BETWEEN TTQ.CREATE_TIME AND CURRENT TIMESTAMP
GROUP BY STOCK;

## Migrating Tables to use Time-Travel Query   

Existing tables can be migrated to use time travel query by altering the table to include the additional columns needed. Usually there is one column in the table that would record the time that the record was created (or updated). This would become the basis for the `SYS_START` column when tracking history values.

### Create the STOCK table without Time-Travel Query Columns   

This SQL will recreate the `STOCK` table but without the columns used for tracking history. We do need one column that records the time that the transaction was done.

In [ ]:
%%sql -q
DROP TABLE TTQ.STOCKS;

CREATE TABLE TTQ.STOCKS
  (
  STOCK      CHAR(3) NOT NULL,
  PRICE      INT NOT NULL,
  QUANTITY   INT NOT NULL,
  TX_DATE    TIMESTAMP(12) NOT NULL
  );

### Generate Data for the STOCK table   

This initial insert sets up the `STOCKS` that will be tracked in this system. In this example we add the physical date to the transaction instead of using the DEFAULT values.

In [ ]:
%%sql
CREATE OR REPLACE VARIABLE TTQ.CREATE_TIME TIMESTAMP;

SET TTQ.CREATE_TIME = CURRENT TIMESTAMP;

INSERT INTO TTQ.STOCKS(STOCK, PRICE, QUANTITY,TX_DATE)
    VALUES
       ('YYZ',100,100,'2012-01-01'),
       ('GRU',100,100,'2012-01-01'),
       ('LHR',100,100,'2012-01-01'),
       ('FRA',100,100,'2012-01-01'),
       ('MEX',100,100,'2012-01-01'),
       ('SYD',100,100,'2012-01-01'),
       ('ICN',100,100,'2012-01-01'),
       ('NRT',100,100,'2012-01-01'),
       ('SIN',100,100,'2012-01-01'),
       ('HKG',100,100,'2012-01-01');
       
SELECT * FROM TTQ.STOCKS;

### Altering the TTQ.STOCKS table   

In order to turn the `STOCKS` table into a Time-travel query table we must do four things to it:

- Add a `SYS_END` column so that we know how long the record is valid for
- Add a `TRANS_ID` column (hidden)
- Change `TX_DATE` column so that it is always GENERATED by default
- Add a Period identifier to the table.

All of these alters can be done with one `ALTER` statement.

In [ ]:
%%sql
ALTER TABLE TTQ.STOCKS 
  ALTER COLUMN TX_DATE 
        SET GENERATED ALWAYS AS ROW BEGIN
  ADD   COLUMN SYS_END 
        TIMESTAMP(12) GENERATED ALWAYS AS ROW END NOT NULL
  ADD   COLUMN TRANS_ID   
        TIMESTAMP(12) GENERATED ALWAYS AS TRANSACTION START ID IMPLICITLY HIDDEN
  ADD    PERIOD SYSTEM_TIME (TX_DATE, SYS_END)
  ;

A select from the table reveals the new `SYS_END` column while the `TRANS_ID` column is hidden.

In [ ]:
%sql SELECT * FROM TTQ.STOCKS;

Finally we create the `HISTORY` table for the `STOCKS` and link the two tables together. 

In [ ]:
%%sql -q
DROP TABLE TTQ.STOCKS_HISTORY;

CREATE TABLE TTQ.STOCKS_HISTORY
  LIKE TTQ.STOCKS;

ALTER TABLE TTQ.STOCKS ADD VERSIONING
  USE HISTORY TABLE TTQ.STOCKS_HISTORY;   

The following SQL generates a number of transactions against the `STOCK` table to show the history file being generated. The first function that we need to create is a "WAIT" function so that we can delay the record updates slightly.

In [ ]:
%%sql -d
CREATE OR REPLACE PROCEDURE TTQ.WAIT(WAIT INTEGER)
  LANGUAGE SQL
  AUTONOMOUS
BEGIN
  DECLARE START_TIME TIMESTAMP;
  DECLARE END_TIME TIMESTAMP;
  DECLARE INTERVAL BIGINT;

  SET INTERVAL = WAIT * 1000;
  SET START_TIME = CURRENT TIMESTAMP;
  SET END_TIME = START_TIME + INTERVAL MICROSECONDS; 
  
  WHILE (END_TIME > CURRENT TIMESTAMP)  DO
  END WHILE;
END
@

This SQL will generate a number of transactions over 100 days. These transactions are updates only since we typically don't delist stocks (it does happen though!).

In [ ]:
%%sql -d
CREATE OR REPLACE TYPE TTQ.VSTOCKS AS INTEGER ARRAY[10]
@
 
CREATE OR REPLACE PROCEDURE TTQ.GENERATE(IN ROWCOUNT INTEGER)
BEGIN
  DECLARE V_TX     INTEGER;
  DECLARE V_STOCK_PRICES TTQ.VSTOCKS;
  DECLARE V_STOCK  CHAR(3);
  DECLARE V_PRICE  INTEGER;
  DECLARE V_DELTA  INTEGER;
  DECLARE V_AMOUNT INTEGER;
  DECLARE IDX      INTEGER;
  DECLARE QUOTE    CHAR(1) DEFAULT '''';
  DECLARE COMMA    CHAR(1) DEFAULT ',';
  DECLARE STOCKS   VARCHAR(100) DEFAULT 'YYZ,GRU,LHR,FRA,MEX,SYD,ICN,NRT,SIN,HKG,XXX';
  DECLARE TEXT     VARCHAR(255);

  DECLARE STMT     STATEMENT; 

  SET V_TX = 1;
  WHILE V_TX <= 10 DO
    SET V_STOCK_PRICES[V_TX] = 100;
    SET V_TX = V_TX + 1;
  END WHILE;
  
  SET V_TX = 1;
  WHILE V_TX <= ROWCOUNT DO
    SET IDX = INT(RAND()*10)+1;
    IF (IDX > 10) THEN
      SET IDX = 10;
    END IF;
    SET V_STOCK = SUBSTR(STOCKS,(IDX-1)*4+1,3);
    SET V_DELTA = INT(RAND()*3)+1;
    IF (INT(RAND()*10) < 3) THEN 
      SET V_STOCK_PRICES[IDX] = V_STOCK_PRICES[IDX] - V_DELTA;
    ELSE
      SET V_STOCK_PRICES[IDX] = V_STOCK_PRICES[IDX] + V_DELTA;
    END IF;
    SET V_AMOUNT = INT(RAND()*100) + 1;
    SET V_PRICE = V_STOCK_PRICES[IDX];
    SET TEXT = 'UPDATE TTQ.STOCKS' ||
               ' SET ' ||
               '   PRICE     = ' || VARCHAR(V_PRICE) || COMMA || 
               '   QUANTITY  = ' || VARCHAR(V_AMOUNT)||
               ' WHERE ' ||
               '   STOCK     = ' || QUOTE || V_STOCK || QUOTE ;
    PREPARE STMT FROM TEXT;
    EXECUTE STMT;
    COMMIT WORK;
    CALL TTQ.WAIT(5);
    SET V_TX = V_TX + 1;
  END WHILE;
END;
@

CALL TTQ.GENERATE(100)
@

SELECT STOCK, PRICE FROM TTQ.STOCKS
@

What is the maximum ad minimum price of all stocks and the number of trades since we created the table?

In [ ]:
%%sql
SELECT STOCK, MIN(PRICE) AS LOW, MAX(PRICE) AS HIGH, COUNT(*) AS TRADES FROM TTQ.STOCKS
  FOR SYSTEM_TIME BETWEEN TTQ.CREATE_TIME AND CURRENT TIMESTAMP
GROUP BY STOCK;

#  Application Period Temporal Tables   

Now the discussion moves to application-period temporal tables and begins with a brief description of application-period temporal tables (ATTs). Application-period temporal tables allow the storage of time-sensitive data. This makes it possible to store information such as insurance policy terms on different dates. As an example, if Joe Smith purchases an automobile policy on 01/01/2010 with a liability coverage of `$50,000` and then upgrades the policy on 03/01/2011 to increase the liability coverage to `$250,000`, ATTs allow the storage of the automobile insurance with different liability coverage for different date ranges.

A key difference between ATTs and the STTs that were previously discussed is that ATTs require a pair of columns defining the begin time and end time (defined as either a TIMESTAMP or DATE data type) to represent the time range where the data is valid.  These two values are controlled by the user and/or application.  Db2 adds, splits, or deletes rows as needed, automatically and transparently. These time values can be used to model data in the past, present, or future, by restricting queries to specific date ranges and allowing data to be defined as valid for specific dates only.

In addition, Db2 supports constraints automatically to disallow overlapping validity periods for a data record. Another difference from STTs is that ATTs do NOT have a separate history table. All date ranges for a particular record are stored in the base table.

## Creating an Application Period Temporal Table

This `CREATE TABLE` statement will build an application-period temporal table named ATT.TRAVEL.

As shown in the example syntax, two columns (bus_start and bus_end) are defined as DATE NOT NULL and the ATT is activated via the "PERIOD BUSINESS TIME (bus_start, bus_end)" clause.  The bus_start date is inclusive and the bus_end date is exclusive, with the additional restriction that bus_start must be less than the bus_end column.

The additional clause on the PRIMARY KEY clause "BUSINESS_TIME WITHOUT OVERLAPS" enforces period uniqueness within the table.  In other words, two records for the same trip cannot overlap between their bus_start and bus_end times.  So there is only one valid trip_name for each trip during each time period. As an example, for a trip named "Grand Canyon", you could only have one "price" valid for any time period. It would be valid to have a price of `$500` valid for 01/01/2011 to 06/30/2011 and a price of `$550` valid for 06/30/2011 to 12/31/2011.  However, you could not have both `$500` and `$550` valid for a single date (04/01/2011) because of this `WITHOUT OVERLAPS` clause.

**Note:** The bus_end time is exclusive, therefore the bus_end time of 06/30/2011 for Grand Canyon Record 1 does NOT overlap with the bus_start time of 06/30/2011 for Grand Canyon Record 2.

In [ ]:
%%sql -q
DROP TABLE ATT.TRAVEL;

CREATE TABLE ATT.TRAVEL
  (
  TRIP_NAME CHAR(25) NOT NULL,
  DESTINATION CHAR(8) NOT NULL,
  DEPARTURE_DATE DATE NOT NULL,
  PRICE DECIMAL(8,2) NOT NULL, 
  BUS_START DATE NOT NULL,
  BUS_END DATE NOT NULL,
  PERIOD BUSINESS_TIME (BUS_START, BUS_END),
  PRIMARY KEY (TRIP_NAME, BUSINESS_TIME WITHOUT OVERLAPS)
  );

## Inserting and Updating Records in an Application Period Temporal Table    

This SQL will insert a new Manu Wilderness trip into the TRAVEL table. Here there is a difference from the System Temporal tables in the earlier examples. Since ATTs utilize business time, the user/application must provide the bus_start and bus_end date values on the INSERT statement.  As seen in a query of the TRAVEL table, the "Manu Wilderness" record has been added.

**Note:** Similar to STTs, the bus_start column date/timestamp is INCLUSIVE (business time >= bus_start) and the bus_end column date/timestamp is EXCLUSIVE (business time < bus_end).

In [ ]:
%%sql
INSERT INTO ATT.TRAVEL VALUES
  ('Manu Wilderness','Peru',
   '08/02/2011',1500.00,'05/01/2011','01/01/2012');
   
SELECT * FROM ATT.TRAVEL;

This example shows an update to the ATT TRAVEL table that was previously created and had the "Manu Wilderness" trip inserted.  Since the "Manu Wilderness" trip is not generating the bookings expected, it is determined that a special price of `$1000.00` will be offered for the month of June 2011.

The important thing to note in this example is that the previously existing record for "Manu Wilderness" had a bus_start of 05/01/2011 and a bus_end of 01/01/2012.  After the UPDATE statement is executed, there are now 3 records for "Manu Wilderness".  This occurred because the month of June indicated in the special price has a bus_start of 06/01/2011 and a bus_end of 07/01/2011 and this is in the middle of the original record time period (05/01/2011 and 01/01/2012). Therefore, Db2 must split the original time period to allow the special June pricing promotion to be inserted into the "Manu Wilderness" trip information.

Therefore, the first valid time period  record for "Manu Wilderness" is from the original bus_start date of 05/01/2011 to 06/01/2011 (remember bus_end is exclusive), which is the start of the pricing promotion period. Then we have the "Manu Wilderness" second record, from 06/01/2011 to 07/01/2011 to set the time period for the pricing promotion.  Next, we have the third record for "Manu Wilderness", which runs from the end of the pricing promotion (07/01/2011) until the original bus_end date of 01/01/2012.

**Note:** Db2 has maintained the original trip price of `$1500.00` for both the time period in advance of the promotion (05/01/2011 to 06/01/2011) and the time period after the promotion (07/01/2011) until the original bus_end date of 01/01/2012.
  

In [ ]:
%%sql
UPDATE ATT.TRAVEL FOR PORTION OF BUSINESS_TIME FROM '06/01/2011' TO '07/01/2011'
  SET PRICE = 1000.00 WHERE TRIP_NAME = 'Manu Wilderness';
  
SELECT * FROM ATT.TRAVEL;

The pricing promotion update generated the expected results, as the original "Manu Wilderness" trip excursion has sold out.  So another section of the trip needs to be added with a departure date of 11/02/2011 with a bus_start time of 10/01/2011 and a bus_end time of 01/01/2012. However, since the original TRAVEL table was defined with the `PRIMARY KEY…BUSINESS_TIME WITHOUT OVERLAPS` clause, the INSERT fails, as the original "Manu Wilderness" trip had a bus_start of 05/01/2011 and a bus_end of 01/01/2012. This new INSERT overlaps the last three months of the original trip and is therefore rejected by Db2.

This error can easily be remedied by changing the name of the second trip to "Manu Wilderness 2" or another name that makes the "trip_name" unique and therefore does not violate the business time overlap restriction.

In [ ]:
%%sql
INSERT INTO ATT.TRAVEL VALUES 
 ('Manu Wilderness','Peru','11/02/2011',1500.00,'10/01/2011','01/01/2012');

The error from the previous example is corrected by changing the "trip_name" to "Manu Wilderness 2" and trying the `INSERT` operation again.  This time the `PRIMARY KEY` restriction on overlaps is not triggered (as the trip name "Manu Wilderness 2" is unique) and the `INSERT` is successful.

As the table data indicates, a new trip "Manu Wilderness 2" has been inserted into the TRAVEL table.

In [ ]:
%%sql
INSERT INTO ATT.TRAVEL VALUES 
  ('Manu Wilderness 2','Peru','11/02/2011',1500.00,'10/01/2011','01/01/2012');
  
SELECT * FROM ATT.TRAVEL;

## Querying the Contents of an Application Period Temporal Table   

The next set of SQL statements will focus on queries against application-period temporal tables with a few examples based on the information that was previously input into the TRAVEL table.

For these query examples, the current date is 09/01/2011. In the first example, the query is asking the price of the "Manu Wilderness" trip AS OF '06/01/2011'.  If you remember the previous operations against the ATT TRAVEL table, 06/01/2011 is within the time period for the pricing promotion for the "Manu Wilderness" trip, so the expected result of `$1000.00` is returned for the query.

In [ ]:
%%sql
SELECT PRICE FROM ATT.TRAVEL FOR BUSINESS_TIME AS OF '06/01/2011' 
  WHERE TRIP_NAME = 'Manu Wilderness';

The second example is asking for the lowest price of the "Manu Wilderness" trip for the year 2011. A request for the lowest price is indicated via the use of the MIN (minimum) function  on the "price" column. Since the lowest price for the "Manu Wilderness" trip was during the June 2011 pricing promotion and this time period is within 2011, the same price (`$1000.00`) is returned as with the prior query example.

In [ ]:
%%sql
SELECT MIN (PRICE) FROM ATT.TRAVEL 
  FOR BUSINESS_TIME FROM '01/01/2011' TO '01/01/2012' 
  WHERE TRIP_NAME = 'Manu Wilderness';

Finally, for the third query, we want to know ALL trips that were available for booking in October 2011. Since the two trips that the TRAVEL table contains are "Manu Wilderness" with a bus_start of '05/01/2011' and bus_end of '01/01/2012' and "Manu Wilderness 2" with a bus_start of '10/01/2011' and bus_end of '01/01/2012', both trips were eligible for booking during the month of October (ignoring the fact that the departure date for "Manu Wilderness" was 08/02/2011). So Db2 returns both the "Manu Wilderness" and "Manu Wilderness 2" trip names as query results.

In [ ]:
%%sql
SELECT TRIP_NAME FROM ATT.TRAVEL
  FOR BUSINESS_TIME BETWEEN '10/01/2011' AND '11/01/2011';

## Deleting Records from an Application Period Temporal Table   

The next example will focus on DELETE operations against the ATT TRAVEL table used in the prior ATT examples. Due to a mudslide tragedy that has befallen the "Manu Wilderness" lodge, all trips from 09/15/2011 until further notice have to be removed from the booking inventory.  As a result, all records are going to be deleted from the TRAVEL table that contain "Manu Wilderness" as part (or all) of the "trip_name" column and are contained in the time period from 09/15/2011 (inclusive) to 12/29/9999 (as 12/30/9999 is exclusive).

From the prior TRAVEL table examples, there were three rows for the "Manu Wilderness" trip with the following bus_start, bus_end pairs:

- (05/01/2011, 06/01/2011)
- (06/01/2011, 07/01/2011)
- (07/01/2011, 01/01/2012)

and one row for the "Manu Wilderness 2" trip with the bus_start, bus_end value of (10/01/2011, 01/01/2012).

In comparing the "Manu Wilderness" trip records to the DELETE time period specification, only part of the third record falls within the designated DELETE times FROM 09/15/2011 to 12/30/9999. Remembering that the bus_end date is exclusive, record #3 of the "Manu Wilderness" has its bus_end column updated from the original bus_end of 01/01/2012 to the first date in the DELETE range of 09/15/2011. Since both the bus_start (10/01/2011) and bus_end (01/01/2012) dates of the "Manu Wilderness 2" record are contained in the DELETE time period specification, the entire "Manu Wilderness 2" trip is deleted from the TRAVEL table.

In [ ]:
%%sql
DELETE FROM ATT.TRAVEL 
  FOR PORTION OF BUSINESS_TIME FROM '09/15/2011' TO '12/30/9999' 
  WHERE TRIP_NAME LIKE 'Manu Wilderness%';

SELECT * FROM ATT.TRAVEL;

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]